# Double Selection et Sample-splitting, simulations

Date: 4 janvier 2018

@author: jeremlhour

In [ ]:
rm(list=ls())

##############################
##############################
### PACKAGES AND FUNCTIONS ###
##############################
##############################

library("ggplot2")
library("gridExtra")
library("glmnet")
library("MASS")

### Load user-defined functions
source("src/DataSim.R")

# 1. Parameters

In [ ]:
set.seed(99999)

R = 100 # nb simulations
n = 200   # sample size
p = 300   # nb variables
K = 5     # nb folds
tau = .5  # Treatment effect

split = runif(n)
cvgroup = as.numeric(cut(split,quantile(split,probs = seq(0, 1, 1/K)),include.lowest = T))  

g = .1/log(max(p,n))
lambda = 1.1*qnorm(1-.5*g/p)/sqrt(n) # (theoretical) Lasso penalty level 

# 2. Simulations

In [ ]:
estimate = matrix(ncol=3, nrow=R)
standard_error = matrix(ncol=3, nrow=R)
t_start = Sys.time()
pb = txtProgressBar(min = 0, max = R, initial = 0) 

for(r in 1:R){
  ### GENERATION DES DONNEES
  data = DataSim(n=n, p=p, Ry=.1, Rd=.8, Intercept=F, a=tau)
  X = data$X; y = data$y; d = data$d
  
  ### METHOD 1: Naive selection
  phi = rep(1,p+1); phi[1] = 0 # Do not penalize "d"
  lasso.selec = glmnet(cbind(d,X),y, family="gaussian",alpha=1,penalty.factor=phi,lambda=lambda)
  b.lasso = coef(lasso.selec); b.lasso = b.lasso[-c(1,2)] # on enleve la constante et la variable de traitement
  s.hat = which(b.lasso != 0)
  if(length(s.hat)==0){
    naive.fit = lm(y ~ d)
  } else {
    naive.fit = lm(y ~ d + X[,s.hat])
  }
  
  ### METHOD 2: Double-Selection, no sample-splitting
  # A. Selection sur le Traitement
  treat.selec = glmnet(X,d, family="gaussian",alpha=1,lambda=lambda)
  b.treat = coef(treat.selec); b.treat = b.treat[-1] # on enleve la constante
  S.d = which(b.treat != 0)
  
  # B. Selection sur l'Outcome
  outcome.selec = glmnet(X,y, family="gaussian",alpha=1,lambda=lambda)
  b.outcome = coef(outcome.selec); b.outcome = b.outcome[-1] # on enleve la constante
  S.y = which(b.outcome != 0)
  
  # C. Calcul de l'estimateur de double selection
  s.hat = union(S.y,S.d)
  if(length(s.hat)==0){
    DS.fit = lm(y ~ d)
  } else {
    DS.fit = lm(y ~ d + X[,s.hat])
  }
  
  # D. Compute sd
  if(length(s.hat)==0){
    treat.fit = lm(d ~ 1)
  } else {
    treat.fit = lm(d ~ X[,s.hat])
  }
  sigmaNum = sum(treat.fit$residuals^2*DS.fit$residuals^2) /(n - length(s.hat) - 1)
  sigmaDenom = sum(treat.fit$residuals^2) / n
  stdev_dbs = sqrt( sigmaNum / sigmaDenom^2) / sqrt(n)
  
  
  ### METHOD 3: Double selection et sample-splitting
  theta = vector(length=K)
  sigma = matrix(nrow=K,ncol=2)
  for(k in 1:K){
    Ik  = (cvgroup==k) # separer les donnees
    NIk = (cvgroup!=k)
    
    # 0. Ajustement du lambda
    gstar = .1/log(max(p,sum(NIk)))
    lambdastar = 1.1*qnorm(1-.5*g/p)/sqrt(sum(NIk)) # Lasso penalty level
    
    # Abis. Selection sur le traitement
    treat.selec = glmnet(X[NIk,],d[NIk], family="gaussian",alpha=1,lambda=lambdastar)
    b.treat = coef(treat.selec); b.treat = b.treat[-1] # on enleve la constante
    S.d = which(b.treat != 0)  
    
    # Bbis. Selection on Outcome
    outcome.selec = glmnet(X[NIk,],y[NIk], family="gaussian",alpha=1,lambda=lambdastar)
    b.outcome = coef(outcome.selec); b.outcome = b.outcome[-1] # on enleve la constante
    S.y = which(b.outcome != 0)
    
    # Cbis. Compute Post-Double-Selection
    s.hat = union(S.y,S.d)
    if(length(s.hat)==0){
      outcomePL = lm(y[NIk] ~ 1)
      treatPL = lm(d[NIk] ~ 1)
    } else {
      outcomePL = lm(y[NIk] ~ X[NIk,s.hat])
      treatPL = lm(d[NIk] ~ X[NIk,s.hat])
    }
    
    # D. Target param on left-out sample
    ytilde = y[Ik] - cbind(rep(1,sum(Ik)),X[Ik,s.hat])%*%coef(outcomePL)
    dtilde = d[Ik] - cbind(rep(1,sum(Ik)),X[Ik,s.hat])%*%coef(treatPL)
    Ikfit = lm(ytilde ~ dtilde)
    
    theta[k] = Ikfit$coef['dtilde']
    sigma[k,] = c(sum(dtilde^2*ytilde^2),
                  sum(dtilde^2))
  }
  # E. Compute standard error for sample-splitting
  sigmaNum = sum(sigma[,1]) / n
  sigmaDenom = sum(sigma[,2]) / n
  stdev_dbs_ss = sqrt(sigmaNum / sigmaDenom^2) / sqrt(n)
  
  
  ### COLLECTING RESULTS
  estimate[r,] = c(naive.fit$coef['d'],
                   DS.fit$coef['d'],
                   mean(theta))
  standard_error[r,] =  c(summary(naive.fit)$coefficients['d',"Std. Error"],
                          stdev_dbs,
                          stdev_dbs_ss)
  
  setTxtProgressBar(pb,r)
}
close(pb)
print(Sys.time()-t_start)

# 3. Compute statistics

In [ ]:
StatDisplay = data.frame()
StatDisplay[1:3, "Bias"] = apply(estimate-tau,2,mean)
StatDisplay[1:3, "RMSE"] = sqrt(apply((estimate-tau)^2,2,mean))
borne_sup = estimate + qnorm(.975) * standard_error - tau
borne_inf = estimate - qnorm(.975) * standard_error - tau
StatDisplay[1:3, "Coverage rate"] = apply((borne_sup>0)*(borne_inf<0),2,mean,na.rm=T)

row.names(StatDisplay) = c("Naive", "Immunized", "Immunized, Cross-fitted")
print(round(StatDisplay, digits=3))

# 4. Plots

In [ ]:
id = c(mapply(function(x) rep(x,R),1:3))
val = c(estimate)-tau
data_res = data.frame(val = val, model = id)

M = max(abs(quantile(estimate,.01,na.rm=T)),abs(quantile(estimate,.99,na.rm=T)))
lb = -1.1*M; ub = 1.1*M

get.plot <- function(data,modelS,title="A Title",s){
  plot_res <- ggplot(subset(data, (model==modelS)), aes(x=val)) + 
    geom_histogram(binwidth = .02, alpha=1, position='identity',fill="grey", aes(y = ..density..)) +
    scale_x_continuous(limits=c(lb,ub), name="") +
    ggtitle(title) + 
    stat_function(fun = dnorm, args=list(mean=0, sd=s), colour="black", size=0.5, linetype = "dashed") +
    theme(
      panel.grid.major = element_blank(),
      panel.grid.minor = element_blank(),
      panel.background = element_blank(),
      plot.title = element_text(lineheight=.8, face="bold"),
      legend.position="none",
      axis.text=element_text(size=12)
    )
  return(plot_res)
}
              
options(repr.plot.width=20, repr.plot.height=8)
grid.arrange(
    get.plot(data_res,1,"Naive Post-Selec", mean(standard_error[,2])),
    get.plot(data_res,2,"Double-Selec", mean(standard_error[,2])),
    get.plot(data_res,3,"Double-Selec, Cross-fitting", mean(standard_error[,2])),
    ncol=3
)

In [ ]:
jpeg("plots/double-selec-simulations.jpg", width=1000, height=400)
grid.arrange(
    get.plot(data_res,1,"Naive Post-Selec", mean(standard_error[,2])),
    get.plot(data_res,2,"Double-Selec", mean(standard_error[,2])),
    get.plot(data_res,3,"Double-Selec, Cross-fitting", mean(standard_error[,2])),
    ncol=3
)
dev.off()